In [ ]:
import numpy as np
import math

def velocity_operator_x(nx, ny, v, theta):
    """
    Construct the x-component of the velocity operator.
    
    The velocity operator is v_x = ∂H/∂k_x for the kinetic part of H.
    
    Parameters:
    -----------
    nx, ny : int
        Lattice dimensions
    v : float
        Fermi velocity parameter
    theta : float
        Twist angle in radians
    
    Returns:
    --------
    V_x : ndarray
        Velocity operator in x-direction
    """
    n = (nx + 1) * (ny + 1)
    dim = 4 * n
    V_x = np.zeros((dim, dim), dtype=complex)
    
    # Rotation factors
    cos_half = math.cos(theta / 2)
    sin_half = math.sin(theta / 2)
    rot_plus = cos_half - 1j * sin_half   # R(+θ/2)
    rot_minus = cos_half + 1j * sin_half  # R(-θ/2)
    
    for l in range(nx + 1):
        for m in range(ny + 1):
            idx = l * (ny + 1) + m
            
            # Layer 1: v_x = -v * R(+θ/2) for off-diagonal
            V_x[idx, idx + n] = -v * rot_plus
            V_x[idx + n, idx] = -v * rot_minus
            
            # Layer 2: v_x = -v * R(-θ/2) for off-diagonal
            V_x[idx + 2*n, idx + 3*n] = -v * rot_minus
            V_x[idx + 3*n, idx + 2*n] = -v * rot_plus
    
    return V_x


def velocity_operator_y(nx, ny, v, theta):
    """
    Construct the y-component of the velocity operator.
    
    The velocity operator is v_y = ∂H/∂k_y for the kinetic part of H.
    
    Parameters:
    -----------
    nx, ny : int
        Lattice dimensions
    v : float
        Fermi velocity parameter
    theta : float
        Twist angle in radians
    
    Returns:
    --------
    V_y : ndarray
        Velocity operator in y-direction
    """
    n = (nx + 1) * (ny + 1)
    dim = 4 * n
    V_y = np.zeros((dim, dim), dtype=complex)
    
    # Rotation factors
    cos_half = math.cos(theta / 2)
    sin_half = math.sin(theta / 2)
    rot_plus = cos_half - 1j * sin_half   # R(+θ/2)
    rot_minus = cos_half + 1j * sin_half  # R(-θ/2)
    
    for l in range(nx + 1):
        for m in range(ny + 1):
            idx = l * (ny + 1) + m
            
            # Layer 1: v_y = i*v * R(+θ/2) for off-diagonal
            V_y[idx, idx + n] = 1j * v * rot_plus
            V_y[idx + n, idx] = -1j * v * rot_minus
            
            # Layer 2: v_y = i*v * R(-θ/2) for off-diagonal
            V_y[idx + 2*n, idx + 3*n] = 1j * v * rot_minus
            V_y[idx + 3*n, idx + 2*n] = -1j * v * rot_plus
    
    return V_y


def normalize_eigenvectors(eigenvectors):
    """
    Normalize the columns of the eigenvector matrix.
    
    Parameters:
    -----------
    eigenvectors : ndarray
        Matrix where each column is an eigenvector
    
    Returns:
    --------
    normalized : ndarray
        Normalized eigenvectors
    """
    dim = eigenvectors.shape[0]
    normalized = np.zeros_like(eigenvectors)
    
    for b in range(dim):
        # Compute norm of column b
        norm = np.linalg.norm(eigenvectors[:, b])
        normalized[:, b] = eigenvectors[:, b] / norm
    
    return normalized


def velocity_matrix_element(eigenvectors, velocity_operator, n1, n2):
    """
    Compute <n1|V|n2> where V is a velocity operator.
    
    Parameters:
    -----------
    eigenvectors : ndarray
        Normalized eigenvector matrix (columns are eigenvectors)
    velocity_operator : ndarray
        Velocity operator matrix
    n1, n2 : int
        Band indices
    
    Returns:
    --------
    result : complex
        Matrix element <n1|V|n2>
    """
    # Extract eigenvectors for bands n1 and n2
    psi_n1 = eigenvectors[:, n1]
    psi_n2 = eigenvectors[:, n2]
    
    # Compute <n1|V|n2> = psi_n1^† · V · psi_n2
    result = np.dot(np.conj(psi_n1), np.dot(velocity_operator, psi_n2))
    
    return result


def compute_qgt_element(eigenvectors, eigenvalues, velocity_ops, n1, 
                        direction1, direction2, lattice_constant):
    """
    Compute elements of the Quantum Geometric Tensor (QGT).
    
    QGT consists of:
    - Quantum metric: g_ij = Re[sum_{n≠m} <n|v_i|m><m|v_j|n> / (E_m - E_n)²]
    - Berry curvature: Ω_ij = -2 Im[sum_{n≠m} <n|v_i|m><m|v_j|n> / (E_m - E_n)²]
    
    Parameters:
    -----------
    eigenvectors : ndarray
        Normalized eigenvector matrix
    eigenvalues : ndarray
        Energy eigenvalues
    velocity_ops : dict
        Dictionary with 'x' and 'y' velocity operators
    n1 : int
        Band index of interest
    direction1, direction2 : str
        'x' or 'y' for the tensor indices
    lattice_constant : float
        Lattice constant for proper units
    
    Returns:
    --------
    g : float
        Quantum metric element g_{direction1,direction2}^{n1}
    omega : float
        Berry curvature element Ω_{direction1,direction2}^{n1}
    """
    dim = len(eigenvalues)
    
    # Get velocity operators for the specified directions
    V1 = velocity_ops[direction1]
    V2 = velocity_ops[direction2]
    
    sum_val = 0.0 + 0.0j
    
    for n2 in range(dim):
        if n2 != n1:
            # Energy difference
            delta_E = eigenvalues[n1] - eigenvalues[n2]
            
            if abs(delta_E) > 1e-10:  # Avoid division by zero
                # Compute <n1|V1|n2>
                v1_element = velocity_matrix_element(eigenvectors, V1, n1, n2)
                
                # Compute <n2|V2|n1>
                v2_element = velocity_matrix_element(eigenvectors, V2, n2, n1)
                
                # Add to sum
                sum_val += v1_element * v2_element / (delta_E ** 2)
    
    # Extract quantum metric and Berry curvature
    g = np.real(sum_val)
    omega = -2 * np.imag(sum_val)
    
    # Scale by lattice constant squared
    a_sq = lattice_constant ** 2
    
    return g * a_sq, omega * a_sq


def compute_qgt_two_band(eigenvectors, eigenvalues, velocity_ops, n1, 
                         direction1, direction2, lattice_constant, nx, ny):
    """
    Compute QGT elements using two-band approximation.
    
    Only includes contributions from the two bands closest to the Fermi level.
    
    Parameters:
    -----------
    eigenvectors : ndarray
        Normalized eigenvector matrix
    eigenvalues : ndarray
        Energy eigenvalues
    velocity_ops : dict
        Dictionary with 'x' and 'y' velocity operators
    n1 : int
        Band index of interest
    direction1, direction2 : str
        'x' or 'y' for the tensor indices
    lattice_constant : float
        Lattice constant for proper units
    nx, ny : int
        Lattice dimensions (to determine middle bands)
    
    Returns:
    --------
    g : float
        Quantum metric element (two-band approximation)
    omega : float
        Berry curvature element (two-band approximation)
    """
    # Identify the two middle bands (closest to Fermi level)
    n_half = 2 * (nx + 1) * (ny + 1)
    band_list = [n_half, n_half - 1]
    
    # Get velocity operators
    V1 = velocity_ops[direction1]
    V2 = velocity_ops[direction2]
    
    sum_val = 0.0 + 0.0j
    
    for n2 in band_list:
        if n2 != n1:
            # Energy difference
            delta_E = eigenvalues[n1] - eigenvalues[n2]
            
            if abs(delta_E) > 1e-10:
                # Compute matrix elements
                v1_element = velocity_matrix_element(eigenvectors, V1, n1, n2)
                v2_element = velocity_matrix_element(eigenvectors, V2, n2, n1)
                
                # Add to sum
                sum_val += v1_element * v2_element / (delta_E ** 2)
    
    # Extract components
    g = np.real(sum_val)
    omega = -2 * np.imag(sum_val)
    
    # Scale by lattice constant
    a_sq = lattice_constant ** 2
    
    return g * a_sq, omega * a_sq
